In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine, text
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

def generate_car_sales_data(num_records=100000):
    """Generate synthetic car sales data with more realistic complexity, randomness, and external factors."""
    regions = {
        "North Jersey": "Wealthiest",
        "Central Jersey": "Middle Class",
        "South Jersey": "Lower Income"
    }

    region_probs = [0.3, 0.4, 0.3]
    region_choices = np.random.choice(list(regions.keys()), size=num_records, p=region_probs)

    car_models = {
        "Corolla": {"mpg": 30, "powertrain": "Gasoline", "safety_rating": 4},
        "Camry": {"mpg": 28, "powertrain": "Hybrid", "safety_rating": 5},
        "RAV4": {"mpg": 25, "powertrain": "Hybrid", "safety_rating": 5},
        "Highlander": {"mpg": 22, "powertrain": "Gasoline", "safety_rating": 5},
        "Tacoma": {"mpg": 20, "powertrain": "Gasoline", "safety_rating": 4},
        "Sienna": {"mpg": 21, "powertrain": "Gasoline", "safety_rating": 5},
        "Prius": {"mpg": 50, "powertrain": "Hybrid", "safety_rating": 5},
        "Sequoia": {"mpg": 17, "powertrain": "Gasoline", "safety_rating": 5}
    }

    car_model_msrp = {
        "Corolla": 22000,
        "Camry": 26000,
        "RAV4": 28000,
        "Highlander": 39000,
        "Tacoma": 31500,
        "Sienna": 37000,
        "Prius": 27000,
        "Sequoia": 61275
    }

    car_model_popularity = {
        "Corolla": 0.2381,
        "Camry": 0.1905,
        "RAV4": 0.1429,
        "Highlander": 0.1429,
        "Tacoma": 0.0952,
        "Sienna": 0.0952,
        "Prius": 0.0476,
        "Sequoia": 0.0476
    }

    total_prob = sum(car_model_popularity.values())
    car_model_popularity = {k: v / total_prob for k, v in car_model_popularity.items()}

    promotions = [0, 5, 10, 15, 20]
    end_date = datetime.now()
    start_date = end_date - timedelta(days=3 * 365)
    all_dates = [start_date + timedelta(days=x) for x in range((end_date - start_date).days)]
    np.random.seed(42)

    model_choices = np.random.choice(list(car_model_popularity.keys()),
                                     size=num_records,
                                     p=list(car_model_popularity.values()))

    chosen_msrp = np.array([car_model_msrp[m] for m in model_choices])

    region_price_factor = {
        "North Jersey": 1.10,
        "Central Jersey": 1.00,
        "South Jersey": 0.90
    }
    region_factors = np.array([region_price_factor[r] for r in region_choices])

    price_variation = np.random.uniform(-0.15, 0.15, num_records)
    prices = (chosen_msrp * (1 + price_variation) * region_factors).round(2)

    chosen_dates = np.random.choice(all_dates, num_records)
    months = np.array([d.month for d in chosen_dates])

    interest_rate = 3 + 1 * np.sin(2 * np.pi * months / 12) + np.random.normal(0, 0.15, num_records)
    interest_rate = np.round(interest_rate, 2)
    employment_rate = 5 - 0.1 * (interest_rate - 3) + np.random.normal(0, 0.12, num_records)
    employment_rate = np.round(employment_rate, 2)

    monthly_competitor_shock = {m: np.random.normal(0, 300) for m in range(1,13)}
    competitor_price = prices + np.random.normal(0, 1000, num_records) + np.array([monthly_competitor_shock[m] for m in months])
    competitor_price = np.round(competitor_price, 2)

    base_seasonal_multipliers = {
        "Winter": 0.90,
        "Spring": 1.00,
        "Summer": 1.05,
        "Fall":   1.00
    }
    seasons_map = {
        12: "Winter",1: "Winter",2: "Winter",
        3: "Spring",4: "Spring",5: "Spring",
        6: "Summer",7: "Summer",8: "Summer",
        9: "Fall", 10: "Fall", 11: "Fall"
    }

    for s in base_seasonal_multipliers:
        base_seasonal_multipliers[s] += np.random.normal(0, 0.05)
        base_seasonal_multipliers[s] = max(0.5, min(1.5, base_seasonal_multipliers[s]))

    season = np.array([seasons_map[m] for m in months])

    base_promotion_prob = {
        "Winter": [0.22,0.22,0.19,0.18,0.19],
        "Spring": [0.22,0.21,0.2,0.19,0.18],
        "Summer": [0.2,0.21,0.21,0.19,0.19],
        "Fall":   [0.21,0.21,0.2,0.19,0.19]
    }
    region_promo_boost = {
        "North Jersey": 0.95,
        "Central Jersey": 1.00,
        "South Jersey": 1.05
    }

    chosen_promotions = []
    for i in range(num_records):
        s = season[i]
        r = region_choices[i]
        weights = np.array(base_promotion_prob[s]) * region_promo_boost[r]
        weights = weights / weights.sum()
        chosen_promotions.append(np.random.choice(promotions, p=weights))
    chosen_promotions = np.array(chosen_promotions)

    trade_in_discount = np.where(prices > 40000,
                                 np.random.uniform(3000,7000,num_records),
                                 np.where(prices > 30000,
                                          np.random.uniform(1500,4000,num_records),
                                          np.random.uniform(500,2000,num_records)))
    trade_in_discount = np.round(trade_in_discount, 2)
    final_price = prices - trade_in_discount

    external_shock = {m: np.random.normal(0, 2) for m in range(1,13)}
    shock_factor = np.array([external_shock[m] for m in months])

    region_qty_factor = {
        "North Jersey": 0.95,
        "Central Jersey": 1.00,
        "South Jersey": 1.05
    }
    region_factor = np.array([region_qty_factor[r] for r in region_choices])
    season_factor = np.array([base_seasonal_multipliers[s] for s in season])

    base_qty = 10
    price_factor = -2 * (final_price / 10000.0)
    promo_factor = 1 + (chosen_promotions / 150.0)

    quantity_sold = (base_qty + price_factor) * promo_factor * season_factor * region_factor
    quantity_sold = quantity_sold + shock_factor
    quantity_sold += np.random.normal(0, 1.5, num_records)
    quantity_sold = np.maximum(1, quantity_sold).round().astype(int)

    data = {
        'sale_id': range(1, num_records + 1),
        'date': chosen_dates,
        'region': region_choices,
        'car_model': model_choices,
        'quantity_sold': quantity_sold,
        'price': prices,
        'promotion': chosen_promotions,
        'employment_rate': employment_rate,
        'interest_rate': interest_rate,
        'age': np.random.randint(20, 80, num_records),
        'competitor_price': competitor_price,
        'trade_in_discount': trade_in_discount,
        'final_price': final_price
    }

    df = pd.DataFrame(data)
    df['region_tier'] = df['region'].map(regions)
    df['age_group'] = df['age'].apply(lambda x: "Young" if 20 <= x <= 35 else ("Mid-Age" if 36 <= x <= 50 else "Older"))
    df['mpg'] = df['car_model'].apply(lambda x: car_models[x]['mpg'])
    df['powertrain'] = df['car_model'].apply(lambda x: car_models[x]['powertrain'])
    df['safety_rating'] = df['car_model'].apply(lambda x: car_models[x]['safety_rating'])
    df['msrp'] = [car_model_msrp[m] for m in df['car_model']]
    df['relative_msrp_diff'] = (df['final_price'] - df['msrp']) / df['msrp']
    df['month'] = df['date'].dt.month
    df['season'] = df['month'].apply(lambda m: seasons_map[m])
    df['is_weekend'] = df['date'].apply(lambda d: 1 if d.weekday() >= 5 else 0)
    df['price_difference'] = df['final_price'] - df['competitor_price']

    return df

def generate_car_sales_data_with_missing_values(num_records=100000, missing_rate=0.1):
    df = generate_car_sales_data(num_records)
    target_columns = ['age', 'employment_rate', 'interest_rate', 'trade_in_discount', 'competitor_price']
    for col in target_columns:
        missing_indices = np.random.choice(df.index, size=int(len(df) * missing_rate), replace=False)
        df.loc[missing_indices, col] = np.nan
    return df

def impute_missing_values(df):
    df['age'] = df['age'].fillna(df['age'].mean())
    df['employment_rate'] = df['employment_rate'].fillna(df['employment_rate'].mean())
    df['interest_rate'] = df['interest_rate'].fillna(df['interest_rate'].mean())
    df['trade_in_discount'] = df['trade_in_discount'].fillna(0)
    df['competitor_price'] = df['competitor_price'].fillna(method='ffill')
    return df

def handle_outliers(df):

    def handle_outliers_iqr(df, columns):
        for col in columns:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            df[col] = np.clip(df[col], lower_bound, upper_bound)
        return df

    features_to_check = ['final_price', 'quantity_sold', 'employment_rate', 'interest_rate']
    df = handle_outliers_iqr(df, features_to_check)

    df['final_price'] = np.log1p(df['final_price'])
    df['quantity_sold'] = np.log1p(df['quantity_sold'])

    return df




def basic_data_inspection(df):
    print(df.head())
    print(df.tail())
    print("\nDescriptive Statistics:")
    print(df.describe())
    print("\nShape of the DataFrame:")
    print(df.shape)

def exploratory_data_analysis(df):
    print("\nExploratory Data Analysis")
    numerical_features = ['final_price', 'quantity_sold', 'employment_rate', 'interest_rate', 'trade_in_discount']
    for feature in numerical_features:
        plt.figure(figsize=(8, 5))
        sns.histplot(df[feature], kde=True, bins=30)
        plt.title(f"Distribution of {feature}")
        plt.xlabel(feature)
        plt.ylabel("Frequency")
        plt.grid()
        plt.show()

    corr_cols = ['final_price', 'quantity_sold', 'employment_rate', 'interest_rate', 'trade_in_discount', 'relative_msrp_diff']
    plt.figure(figsize=(10, 6))
    corr = df[corr_cols].corr()
    sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f", square=True)
    plt.title("Correlation Heatmap")
    plt.show()

    plt.figure(figsize=(8, 5))
    sns.scatterplot(x='final_price', y='quantity_sold', data=df, hue='region', alpha=0.6)
    plt.title("Final Price vs. Quantity Sold")
    plt.xlabel("Final Price")
    plt.ylabel("Quantity Sold")
    plt.legend(title="Region")
    plt.grid()
    plt.show()

    sales_by_region = df.groupby('region')['quantity_sold'].sum().reset_index()
    plt.figure(figsize=(8, 5))
    sns.barplot(x='region', y='quantity_sold', data=sales_by_region, palette='pastel')
    plt.title("Total Sales by Region")
    plt.xlabel("Region")
    plt.ylabel("Total Quantity Sold")
    plt.grid()
    plt.show()


def insert_data_sqlalchemy(df, db_name='car_sales.db'):
    engine = create_engine(f"sqlite:///{db_name}", echo=False)
    df.to_sql('car_sales', engine, if_exists='replace', index=False)

    with engine.connect() as conn:
        result = conn.execute(text("SELECT * FROM car_sales WHERE region='North Jersey' LIMIT 5;"))
        rows = result.fetchall()
        print("\nSample rows from 'North Jersey':")
        for r in rows:
            print(r)

        update_result = conn.execute(text("UPDATE car_sales SET promotion=20 WHERE region='South Jersey' AND promotion<20;"))
        print(f"\nNumber of rows updated (promotion set to 20 in South Jersey): {update_result.rowcount}")

        delete_result = conn.execute(text("DELETE FROM car_sales WHERE quantity_sold < 2;"))
        print(f"Number of rows deleted (quantity_sold < 2): {delete_result.rowcount}")

        avg_sales = conn.execute(text("SELECT AVG(quantity_sold) FROM car_sales;")).scalar()
        print(f"\nAverage quantity sold after updates and deletions: {avg_sales}")

def train_linear_regression_model(df):
    print("\n--- Training Linear Regression Model (Linear) ---")
    predictors = [
        'promotion',
        'employment_rate',
        'interest_rate',
        'region',
        'car_model',
        'final_price',
        'relative_msrp_diff',
        'is_weekend',
        'season'
    ]
    target = 'quantity_sold'

    df_encoded = pd.get_dummies(df[predictors], columns=['region', 'car_model', 'season'], drop_first=True)
    df_encoded[target] = df[target]

    X = df_encoded.drop(target, axis=1)
    y = df_encoded[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Mean Squared Error (MSE): {mse:.2f}")
    print(f"R² Score: {r2:.2f}")

    feature_importance = pd.DataFrame({
        'Feature': X_train.columns,
        'Coefficient': model.coef_
    }).sort_values(by='Coefficient', ascending=False)

    print("\nFeature Importance (Linear):")
    print(feature_importance)

    plt.figure(figsize=(8, 5))
    plt.scatter(y_test, y_pred, alpha=0.6, color='blue')
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
    plt.title("True vs. Predicted Quantity Sold (Linear)")
    plt.xlabel("True Quantity Sold")
    plt.ylabel("Predicted Quantity Sold")
    plt.grid()
    plt.show()

    return model

def train_random_forest_model(df):
    print("\nRandom Forest Model")
    predictors = [
        'promotion',
        'employment_rate',
        'interest_rate',
        'region',
        'car_model',
        'final_price',
        'relative_msrp_diff',
        'is_weekend',
        'season'
    ]
    target = 'quantity_sold'

    df_encoded = pd.get_dummies(df[predictors], columns=['region', 'car_model', 'season'], drop_first=True)
    df_encoded[target] = df[target]

    X = df_encoded.drop(target, axis=1)
    y = df_encoded[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    rf_model = RandomForestRegressor(n_estimators=200, random_state=42, max_depth=10)
    rf_model.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Random Forest Mean Squared Error (MSE): {mse:.2f}")
    print(f"Random Forest R² Score: {r2:.2f}")

    feature_importances = pd.DataFrame({
        'Feature': X_train.columns,
        'Importance': rf_model.feature_importances_
    }).sort_values(by='Importance', ascending=False)
    print("\nFeature Importance (Random Forest):")
    print(feature_importances)

    plt.figure(figsize=(8, 5))
    plt.scatter(y_test, y_pred, alpha=0.6, color='green')
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
    plt.title("True vs. Predicted Quantity Sold (Random Forest)")
    plt.xlabel("True Quantity Sold")
    plt.ylabel("Predicted Quantity Sold")
    plt.grid()
    plt.show()

    return rf_model

if __name__ == "__main__":
    df = generate_car_sales_data_with_missing_values(100000, missing_rate=0.1)
    df = impute_missing_values(df)
    basic_data_inspection(df)
    exploratory_data_analysis(df)
    df = handle_outliers(df)


insert_data_sqlalchemy(df)  

linear_model = train_linear_regression_model(df)
rf_model = train_random_forest_model(df)








